# Machine Learning Model Building Pipeline: Big Data, Images and Neural Networks


## Imports

In [5]:
import pandas as pd 
import numpy as np

# navigate folders
from glob import glob
import os

# for plotting 
import matplotlib.pyplot as plt 
%matplotlib inline

# to open the images 
import cv2 

# to display all the columns of the dataframe in the notebook
pd.pandas.set_option('display.max_columns', None)

# data preprocessing

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# evaluate model and separate train and test
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore")

# for the convolutional network
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.optimizers import Adam 
from keras.metrics import categorical_crossentropy 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model 
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint  
from keras.utils import np_utils

## Load Images / Data

In [7]:
# here are all our images
DATA_FOLDER = 'v2-plant-seedlings-dataset'

In [8]:
# each weed class is in a dedicated folder
os.listdir(DATA_FOLDER)

['Black-grass',
 'Charlock',
 'Cleavers',
 'Common Chickweed',
 'Common wheat',
 'Fat Hen',
 'Loose Silky-bent',
 'Maize',
 'nonsegmentedv2',
 'Scentless Mayweed',
 'Shepherds Purse',
 'Small-flowered Cranesbill',
 'Sugar beet']

In [18]:
# let's wal over the directory structure, so we understand hot the images are stored

for class_folder_name in os.listdir(DATA_FOLDER):
    class_folder_name = os.path.join(DATA_FOLDER, class_folder_name)
    for image_path in glob(os.path.join(class_folder_name, "*.png")):
        print(image_path)

v2-plant-seedlings-dataset\Black-grass\1.png
v2-plant-seedlings-dataset\Black-grass\10.png
v2-plant-seedlings-dataset\Black-grass\100.png
v2-plant-seedlings-dataset\Black-grass\101.png
v2-plant-seedlings-dataset\Black-grass\102.png
v2-plant-seedlings-dataset\Black-grass\103.png
v2-plant-seedlings-dataset\Black-grass\104.png
v2-plant-seedlings-dataset\Black-grass\105.png
v2-plant-seedlings-dataset\Black-grass\106.png
v2-plant-seedlings-dataset\Black-grass\107.png
v2-plant-seedlings-dataset\Black-grass\108.png
v2-plant-seedlings-dataset\Black-grass\109.png
v2-plant-seedlings-dataset\Black-grass\11.png
v2-plant-seedlings-dataset\Black-grass\110.png
v2-plant-seedlings-dataset\Black-grass\111.png
v2-plant-seedlings-dataset\Black-grass\112.png
v2-plant-seedlings-dataset\Black-grass\113.png
v2-plant-seedlings-dataset\Black-grass\114.png
v2-plant-seedlings-dataset\Black-grass\115.png
v2-plant-seedlings-dataset\Black-grass\116.png
v2-plant-seedlings-dataset\Black-grass\117.png
v2-plant-seedling

In [26]:
# lets crate a dataframe:
# the datframe stores the path to the image in a one column
# and the class of the weed (the target) in the next column

images_df = []

# navigate within each folder
for class_folder_name in os.listdir(DATA_FOLDER):
    class_folder_path = os.path.join(DATA_FOLDER, class_folder_name)

    # collect every image path
    for image_path in glob(os.path.join(class_folder_path, "*.png")):
        tmp = pd.DataFrame([image_path, class_folder_name]).T
        images_df.append(tmp)

print(images_df)
# concatenate the final df
images_df = pd.concat(images_df, axis=0, ignore_index=True)
images_df.columns = ['image', 'target']
images_df.head(10)

[                                              0            1
0  v2-plant-seedlings-dataset\Black-grass\1.png  Black-grass,                                                0            1
0  v2-plant-seedlings-dataset\Black-grass\10.png  Black-grass,                                                 0            1
0  v2-plant-seedlings-dataset\Black-grass\100.png  Black-grass,                                                 0            1
0  v2-plant-seedlings-dataset\Black-grass\101.png  Black-grass,                                                 0            1
0  v2-plant-seedlings-dataset\Black-grass\102.png  Black-grass,                                                 0            1
0  v2-plant-seedlings-dataset\Black-grass\103.png  Black-grass,                                                 0            1
0  v2-plant-seedlings-dataset\Black-grass\104.png  Black-grass,                                                 0            1
0  v2-plant-seedlings-dataset\Black-grass\105.png  Bl

,image,target
0,v2-plant-seedlings-dataset\Black-grass\1.png,Black-grass
1,v2-plant-seedlings-dataset\Black-grass\10.png,Black-grass
2,v2-plant-seedlings-dataset\Black-grass\100.png,Black-grass
3,v2-plant-seedlings-dataset\Black-grass\101.png,Black-grass
4,v2-plant-seedlings-dataset\Black-grass\102.png,Black-grass
5,v2-plant-seedlings-dataset\Black-grass\103.png,Black-grass
6,v2-plant-seedlings-dataset\Black-grass\104.png,Black-grass
7,v2-plant-seedlings-dataset\Black-grass\105.png,Black-grass
8,v2-plant-seedlings-dataset\Black-grass\106.png,Black-grass
9,v2-plant-seedlings-dataset\Black-grass\107.png,Black-grass


## Data preprocessing 